In [1]:
import soccer_twos

env = soccer_twos.make(render=True,flatten_branched=True)
print("Observation Space: ", env.observation_space.shape)
print("Action Space: ", env.action_space)




[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0


INFO:mlagents_envs.environment:Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0


[INFO] Connected new brain: SoccerTwos?team=1


INFO:mlagents_envs.environment:Connected new brain: SoccerTwos?team=1


[INFO] Connected new brain: SoccerTwos?team=0


INFO:mlagents_envs.environment:Connected new brain: SoccerTwos?team=0


Observation Space:  (336,)
Action Space:  Discrete(27)


In [3]:
print(env.observation_space.shape)
print(env.action_space)

(336,)
Discrete(27)


In [6]:
team0_reward = 0
team1_reward = 0
env.reset()

obs, reward, done, info = env.step(
    {
        0: env.action_space.sample(),
        1: env.action_space.sample(),
        2: env.action_space.sample(),
        3: env.action_space.sample(),
    }
)


print(env.action_space.sample())
print(reward, done['__all__'])


# print(obs_n[0].reshape(1, -1).shape)


11
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0} False


In [6]:
# env.close()

In [7]:
import os
import time
import argparse
import numpy as np
from simple_model import MAModel
from simple_agent import MAAgent
from parl.algorithms import MADDPG
from parl.env.multiagent_env import MAenv
from parl.utils import logger, summary
from gym import spaces

CRITIC_LR = 0.01  # learning rate for the critic model
ACTOR_LR = 0.01  # learning rate of the actor model
GAMMA = 0.95  # reward discount factor
TAU = 0.01  # soft update
BATCH_SIZE = 1024
MAX_STEP_PER_EPISODE = 25  # maximum step per episode
EVAL_EPISODES = 3


# Runs policy and returns episodes' rewards and steps for evaluation
def run_evaluate_episodes(env, agents, eval_episodes):
    eval_episode_rewards = []
    eval_episode_steps = []
    while len(eval_episode_rewards) < eval_episodes:
        obs_n = env.reset()
        done = False
        total_reward = 0
        steps = 0
        while not done and steps < MAX_STEP_PER_EPISODE:
            steps += 1
            action_n = {}

            for agent, obs in zip(agents, obs_n):
                action_n[obs] = np.argmax(agent.sample(obs_n[obs]))
                
            obs_n, reward_n, done_n, _ = env.step(action_n)
            done = all(done_n)
            for i, reward in enumerate(reward_n):
                total_reward += reward_n[i]

        eval_episode_rewards.append(total_reward)
        eval_episode_steps.append(steps)
    return eval_episode_rewards, eval_episode_steps


def run_episode(env, agents):
    obs_n = env.reset()
    done = False
    total_reward = 0
    agents_reward = [0 for _ in range(env.n)]
    steps = 0
    while not done and steps < MAX_STEP_PER_EPISODE:
        steps += 1

        action_n = {}

        for agent, obs in zip(agents, obs_n):
            action_n[obs] = np.argmax(agent.sample(obs_n[obs]))

        next_obs_n, reward_n, done_n, _ = env.step(action_n)

        done = done_n['__all__']

        # store experience
        for i, agent in enumerate(agents):

            agent.add_experience(obs_n[i], action_n[i], reward_n[i],next_obs_n[i], done_n['__all__'])

        # compute reward of every agent
        obs_n = next_obs_n
        for i, reward in enumerate(reward_n):
            total_reward += reward_n[i]
            agents_reward[i] += reward_n[i]

        # show model effect without training
        if False:#args.restore
            continue

        # learn policy
        for i, agent in enumerate(agents):
            critic_loss = agent.learn(agents)

    return total_reward, agents_reward, steps

In [8]:
def main():

    # build agents
    agents = []
    
    players_num = 4

    multi_action_space=[env.action_space,env.action_space,env.action_space,env.action_space]
    multi_obs_shape_n=[336, 336, 336, 336]
    multi_act_shape_n=[27,27,27,27]
    critic_in_dim = sum(multi_obs_shape_n) + sum(multi_act_shape_n)
    
    env.n = 4

    for i in range(players_num):
        
        model = MAModel(multi_obs_shape_n[i], multi_act_shape_n[i], critic_in_dim, False)
        algorithm = MADDPG(
            model,
            agent_index=i,
            act_space=multi_action_space,
            gamma=GAMMA,
            tau=TAU,
            critic_lr=CRITIC_LR,
            actor_lr=ACTOR_LR)
        agent = MAAgent(
            algorithm,
            agent_index=i,
            obs_dim_n=multi_obs_shape_n,
            act_dim_n=multi_act_shape_n,
            batch_size=BATCH_SIZE)
        
        agents.append(agent)
        


        
    if False:#args.restore:
        # restore modle
        for i in range(len(agents)):
            model_file = args.model_dir + '/agent_' + str(i)
            if not os.path.exists(model_file):
                raise Exception(
                    'model file {} does not exits'.format(model_file))
            agents[i].restore(model_file)

    total_steps = 0
    total_episodes = 0
    while total_episodes <= 3000:
        # run an episode
        ep_reward, ep_agent_rewards, steps = run_episode(env, agents)
        summary.add_scalar('train/episode_reward_wrt_episode', ep_reward,
                           total_episodes)
        summary.add_scalar('train/episode_reward_wrt_step', ep_reward,
                           total_steps)
        logger.info(
            'total_steps {}, episode {}, reward {}, agents rewards {}, episode steps {}'
            .format(total_steps, total_episodes, ep_reward, ep_agent_rewards,
                    steps))

        total_steps += steps
        total_episodes += 1

        # evaluste agents
        if total_episodes % 20 == 0:

            eval_episode_rewards, eval_episode_steps = run_evaluate_episodes(
                env, agents, EVAL_EPISODES)
            summary.add_scalar('eval/episode_reward',
                               np.mean(eval_episode_rewards), total_episodes)
            logger.info('Evaluation over: {} episodes, Reward: {}'.format(
                EVAL_EPISODES, np.mean(eval_episode_rewards)))

            # save model
            if False:#not args.restore:
                model_dir = args.model_dir
                os.makedirs(os.path.dirname(model_dir), exist_ok=True)
                for i in range(len(agents)):
                    model_name = '/agent_' + str(i)
                    agents[i].save(model_dir + model_name)

main()

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @tensorboard.py:32] WRN [tensorboard] logdir is None, will save tensorboard files to train_log
View the data using: tensorboard --logdir=train_log --host=192.168.12.58
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 0, episode 0, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 1, episode 1, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 2, episode 2, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 3, episode 3, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 4, episode 4, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 40, episode 40, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 41, episode 41, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 42, episode 42, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 43, episode 43, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 44, episode 44, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:22:59 MainThread @3002647309.py:56] total_steps 45, episode 45, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 80, episode 80, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 81, episode 81, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 82, episode 82, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 83, episode 83, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 84, episode 84, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 85, episode 85, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 120, episode 120, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 121, episode 121, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 122, episode 122, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 123, episode 123, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 124, episode 124, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 125, episode 125, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0,

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 160, episode 160, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 161, episode 161, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 162, episode 162, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 163, episode 163, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 164, episode 164, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:00 MainThread @3002647309.py:56] total_steps 165, episode 165, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0,

...

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2840, episode 2840, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2841, episode 2841, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2842, episode 2842, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2843, episode 2843, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2844, episode 2844, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2845, episode 2845, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2880, episode 2880, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2881, episode 2881, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2882, episode 2882, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2883, episode 2883, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2884, episode 2884, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:26 MainThread @3002647309.py:56] total_steps 2885, episode 2885, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2920, episode 2920, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2921, episode 2921, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2922, episode 2922, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2923, episode 2923, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2924, episode 2924, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2925, episode 2925, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2960, episode 2960, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2961, episode 2961, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2962, episode 2962, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2963, episode 2963, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2964, episode 2964, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 2965, episode 2965, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
{0: 

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
[10-26 15:23:27 MainThread @3002647309.py:56] total_steps 3000, episode 3000, reward 6, agents rewards [0, 1, 2, 3], episode steps 1
